# Basic 3WNN Clustering w/ Gated Labels

In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)
quiet_library(ArchR)

In [ ]:
addArchRThreads(32)
addArchRGenome("hg38")

# Load CD4 Gated Subset

## CD4 Subsets

In [ ]:
cd4_na <- readRDS('../../Seurat_Objects/cd4_naive.rds')

# Add Metadata

In [ ]:
cd4_na <- SetIdent(cd4_na, value = 'pbmc_sample_id')
cd4_na <- RenameIdents(cd4_na,
                           'PB00173-02' = 'Positive', #validated
                           'PB00192-02' = 'Negative', #validated
                           'PB00197-02' = 'Negative', #validated
                           'PB00593-04' = 'Positive', #validated
                           'PB00172-02' = 'Positive',
                           'PB00182-02' = 'Negative',
                           'PB00323-02' = 'Positive',
                           'PB00807-02' = 'Positive',
                           'PB00124-02' = 'Positive',
                           'PB00127-02' = 'Negative',
                           'PB00334-03' = 'Negative',
                           'PB00353-03' = 'Negative',
                           'PB00368-04' = 'Positive',
                           'PB00377-03' = 'Negative',
                           'PB00545-02' = 'Negative',
                           'PB00599-02' = 'Positive')
cd4_na$CMV <- Idents(cd4_na)

In [ ]:
cd4_na <- SetIdent(cd4_na, value = 'pbmc_sample_id')
cd4_na <- RenameIdents(cd4_na,
                           'PB00173-02' = 'UP1006', #validated
                           'PB00192-02' = 'UP1007', #validated
                           'PB00197-02' = 'UP1010', #validated
                           'PB00593-04' = 'BR2002', #validated
                           'PB00172-02' = 'UP1002',
                           'PB00182-02' = 'UP1003',
                           'PB00323-02' = 'BR2004',
                           'PB00807-02' = 'UP1005',
                           'PB00124-02' = 'UP1001',
                           'PB00127-02' = 'UP1004',
                           'PB00334-03' = 'BR2009',
                           'PB00353-03' = 'BR2008',
                           'PB00368-04' = 'BR2005',
                           'PB00377-03' = 'BR2015',
                           'PB00545-02' = 'BR2042',
                           'PB00599-02' = 'BR2052')
cd4_na$subject_id <- Idents(cd4_na)

In [ ]:
cd4_na$age_cmv <- paste0(cd4_na$pediatric_senior,"_CMV-",cd4_na$CMV)

# RNA Clustering

In [ ]:
outlier_genes <- c('AC105402.3','MTRNR2L8','EDA','IFNG-AS1')

In [ ]:
DefaultAssay(cd4_na) <- 'RNA'
cd4_na <- suppressWarnings(SCTransform(cd4_na))

In [ ]:
var_feat_sct <- cd4_na@assays$SCT@var.features
length(var_feat_sct)

In [ ]:
outlier_genes %in% cd4_na@assays$SCT@var.features

In [ ]:
var_feats <- cd4_na@assays$SCT@var.features
var_feats_trim <- var_feats[!(var_feats %in% outlier_genes)]
length(var_feats_trim)

In [ ]:
cd4_na <- suppressWarnings(SCTransform(cd4_na, residual.features = var_feats_trim)) %>% RunPCA()

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
cd4_na <- suppressWarnings(RunHarmony(object = cd4_na, reduction.save = 'rna_harmony',group.by.vars = 'batch_id', reduction = 'pca', 
                                           plot_convergence = T, assay.use = 'SCT'))

In [ ]:
ElbowPlot(cd4_na, ndims = 50)

In [ ]:
cd4_na <- RunUMAP(cd4_na, dims = 1:12, reduction = 'rna_harmony')

## RNA UMAPs

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p1 <- DimPlot(cd4_na, reduction = 'umap', group.by = 'predicted.t_celltype.l2', shuffle = TRUE)
p2 <- DimPlot(cd4_na, reduction = 'umap', group.by = 'pediatric_senior', shuffle = TRUE)
p3 <- DimPlot(cd4_na, reduction = 'umap', group.by = 'age_cmv', shuffle = TRUE)
p4 <- DimPlot(cd4_na, reduction = 'umap', group.by = 'pbmc_sample_id', shuffle = TRUE)
p1 + p2 + p3 + p4

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)
suppressMessages(FeaturePlot(cd4_na, features = c('sct_IFNG-AS1','sct_EDA','sct_SOX4'), 
                             ncol = 3,reduction = 'umap', order = TRUE) & scale_color_viridis())

# ADT Clustering

In [ ]:
adts_to_remove <- c('CD134','CD172a','CD183','CD366','CX3CR1','TCRab','TCRgd','CD137','CD24','CD294','CD304','CD40','CD80','CD86','IgG1-K-Isotype-Control')
adt_mtx <- cd4_na@assays$ADT@counts
length(rownames(adt_mtx))

In [ ]:
idx <- which(rownames(adt_mtx) %in% adts_to_remove)
clean_adt_mtx <- adt_mtx[-idx,]
length(rownames(clean_adt_mtx))

In [ ]:
cd4_na[['clean_adt']] <- CreateAssayObject(clean_adt_mtx)
DefaultAssay(cd4_na) <- 'clean_adt'

In [ ]:
# cluster by ADTs, make sure to save under all new reductions and keys 
VariableFeatures(cd4_na) <- rownames(cd4_na[["clean_adt"]])
cd4_na <- NormalizeData(cd4_na, normalization.method = 'CLR', margin = 2) %>% 
     ScaleData() %>% RunPCA(reduction.name = 'apca', approx = FALSE, reduction.key = 'APC_')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
cd4_na <- suppressWarnings(RunHarmony(object = cd4_na, reduction.save = 'adt_harmony',group.by.vars = 'batch_id', reduction = 'apca', 
                                           plot_convergence = T, assay.use = 'clean_adt'))

In [ ]:
cd4_na <- RunUMAP(cd4_na, reduction = 'adt_harmony', reduction.name = 'adtumap', dims = 1:20)

## ADT UMAPs

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p1 <- DimPlot(cd4_na, reduction = 'adtumap', group.by = 'predicted.t_celltype.l2')
p2 <- DimPlot(cd4_na, reduction = 'adtumap', group.by = 'pediatric_senior')
p3 <- DimPlot(cd4_na, reduction = 'adtumap', group.by = 'age_cmv')
p4 <- DimPlot(cd4_na, reduction = 'adtumap', group.by = 'pbmc_sample_id')
p1 + p2 + p3 + p4

# ATAC Clustering

## Load ArchR Project

In [ ]:
proj <- loadArchRProject(path = '../../PedSen_ATAC/')
proj

## Subset ArchR Project

In [ ]:
bcs <- colnames(cd4_na[['RNA']])
head(bcs)

In [ ]:
idx_new <- which(proj$cellNames_clean %in% bcs)
head(idx_new)

In [ ]:
cells_subset <- proj$cellNames[idx_new]
proj_subset <- proj[cells_subset,]
proj_subset

## ATAC Clustering

In [ ]:
proj_subset <- addIterativeLSI(proj_subset, name = 'IterativeLSI', force = TRUE)

In [ ]:
proj_subset <- addHarmony(proj_subset, reducedDims = 'IterativeLSI', groupBy = 'batch_id', force = TRUE)

In [ ]:
#proj_subset <- addClusters(proj_subset, reducedDims = 'IterativeLSI', name = 'Clusters', force = TRUE, resolution = 0.2)
proj_subset <- addClusters(proj_subset, reducedDims = 'Harmony', name = 'Clusters', force = TRUE, resolution = 0.2)

In [ ]:
#proj_subset <- addUMAP(proj_subset, reducedDims = 'IterativeLSI', name = 'UMAP', force = TRUE)
proj_subset <- addUMAP(proj_subset, reducedDims = 'Harmony', name = 'UMAP', force = TRUE)

In [ ]:
options(repr.plocd4_naidth = 8, repr.plot.height = 8)
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "Clusters", embedding = "UMAP")
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "pediatric_senior", embedding = "UMAP")
plotEmbedding(ArchRProj = proj_subset, colorBy = "cellColData", name = "predicted.t_celltype.l2", embedding = "UMAP")

## Export LSI

In [ ]:
#lsi <- getReducedDims(proj_subset, reducedDims = 'IterativeLSI')
lsi <- getReducedDims(proj_subset, reducedDims = 'Harmony')

## Import LSI to Seurat Object

In [ ]:
rna_metadata <- cd4_na@meta.data
archr_style_barcodes <- paste(rna_metadata$batch_id, "-P1_",rna_metadata$pbmc_sample_id,"#",rownames(rna_metadata), sep = "")
head(archr_style_barcodes)

In [ ]:
# create data frame w/ seurat barcodes and archr style barcodes
bc_df <- data.frame(seurat_bcs = colnames(cd4_na@assays$RNA),
                    archr_style = archr_style_barcodes)

In [ ]:
# Find Intersection & subset LSI
bc_idx <- intersect(rownames(lsi), archr_style_barcodes)

In [ ]:
lsi_subset <- subset(lsi, subset = rownames(lsi) %in% bc_idx)
bc_df_subset <- subset(bc_df, subset = bc_df$archr_style %in% bc_idx)
length(rownames(lsi_subset))
length(rownames(bc_df_subset))

In [ ]:
table(bc_df_subset$archr_style == rownames(lsi_subset))

In [ ]:
lsi_subset <- lsi_subset[match(bc_df_subset$archr_style, rownames(lsi_subset)),]

In [ ]:
table(bc_df_subset$archr_style == rownames(lsi_subset))

In [ ]:
# replace LSI archr barcodes for seurat barcodes
rownames(lsi_subset) <- bc_df_subset$seurat_bcs

In [ ]:
cd4_na <- subset(cd4_na, cells = bc_df_subset$seurat_bcs)

In [ ]:
table(colnames(cd4_na[['RNA']]) == rownames(lsi_subset))

In [ ]:
cd4_na[["lsit"]] <- CreateDimReducObject(embeddings = lsi_subset, key = "lsit_", assay = "Tiles")

## ATAC clustering in Seurat Object

In [ ]:
DefaultAssay(cd4_na) <- 'Tiles'
cd4_na <- RunUMAP(cd4_na, reduction = 'lsit', reduction.name = 'atac_umap', dims = 1:29, verbose = T, reduction.key = 'atacumap_')

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
p1 <- DimPlot(cd4_na, reduction = 'atac_umap', group.by = 'predicted.t_celltype.l2')
p2 <- DimPlot(cd4_na, reduction = 'atac_umap', group.by = 'pediatric_senior')
p3 <- DimPlot(cd4_na, reduction = 'atac_umap', group.by = 'age_cmv')
p4 <- DimPlot(cd4_na, reduction = 'atac_umap', group.by = 'pbmc_sample_id')
p1 + p2 + p3 + p4

# 3WNN Clustering

In [ ]:
cd4_na <- FindMultiModalNeighbors(cd4_na, reduction.list = list("rna_harmony","adt_harmony",'lsit'), dims.list = list(1:8,1:20,1:29), weighted.nn.name = "X3way.weighted.nn",
                                      k.nn = 30, knn.range = 500, prune.SNN = 1/20) #previous 8,10,29

In [ ]:
cd4_na <- RunUMAP(cd4_na, nn.name = "X3way.weighted.nn", reduction.name = "wnn.3.umap", reduction.key = "wnn.3.umap_")

In [ ]:
cd4_na <- FindClusters(cd4_na, graph.name = "wsnn", algorithm = 3, resolution = 0.3, verbose = TRUE)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 15)
p1 <- DimPlot(cd4_na, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.3', label = TRUE)
p2 <- DimPlot(cd4_na, reduction = 'wnn.3.umap', group.by = 'pediatric_senior', shuffle = TRUE, cols = c('#1b9e77','#d95f02'))
p3 <- DimPlot(cd4_na, reduction = 'wnn.3.umap', group.by = 'age_cmv')
p4 <- DimPlot(cd4_na, reduction = 'wnn.3.umap', group.by = 'pbmc_sample_id', shuffle = TRUE)
p1 + p2 + p3 + p4

In [ ]:
sort(table(cd4_na$`wsnn_res.0.3`), decreasing = TRUE)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 12)
suppressMessages(FeaturePlot(cd4_na, features = c('sct_IFNG-AS1','sct_EDA','sct_SOX4','sct_FOXP3'), 
                             ncol = 2,reduction = 'wnn.3.umap', order = TRUE) & scale_color_viridis())

# Clustree

## Setup

In [ ]:
cd4_na <- FindClusters(cd4_na, graph.name = "wsnn", algorithm = 3, resolution = 0.1, verbose = TRUE)

In [ ]:
cd4_na <- FindClusters(cd4_na, graph.name = "wsnn", algorithm = 3, resolution = 0.2, verbose = TRUE)

In [ ]:
cd4_na <- FindClusters(cd4_na, graph.name = "wsnn", algorithm = 3, resolution = 0.4, verbose = TRUE)

In [ ]:
cd4_na <- FindClusters(cd4_na, graph.name = "wsnn", algorithm = 3, resolution = 0.5, verbose = TRUE)

In [ ]:
cd4_na <- FindClusters(cd4_na, graph.name = "wsnn", algorithm = 3, resolution = 0.8, verbose = TRUE)

In [ ]:
library(clustree)

## Plots

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
clustree(cd4_na@meta.data, prefix = "wsnn_res.")

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 15)
p1 <- DimPlot(cd4_na, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.1', label = TRUE)
p2 <- DimPlot(cd4_na, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.2', shuffle = TRUE, label = TRUE)
p3 <- DimPlot(cd4_na, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.3', shuffle = TRUE, label = TRUE)
p4 <- DimPlot(cd4_na, reduction = 'wnn.3.umap', group.by = 'wsnn_res.0.5', shuffle = TRUE, label = TRUE)
p1 + p2 + p3 + p4

In [ ]:
table(cd4_na$wsnn_res.0.3)

# Save 

In [ ]:
saveRDS(cd4_na, file = 'cd4_naive_processed.rds')

In [ ]:
sessionInfo()